# Time Evolving Block Decimation (TEBD)

In this notebook, we will implement the Time-Evolving Block Decimation (TEBD) algorithm seen in the course. TEBD is a relatively simple yet powerful algorithm for computing quantum dynamics with MPS. We will use the transverse field Ising model as an example.

## Table of Contents

1. Transverse Field Ising Model
2. TEBD Algorithm Outline
3. Implementing TEBD Step by Step
4. Exercises

## 1. Transverse Field Ising Model

The Hamiltonian for the transverse field Ising model is given by:

$$
H = -J \sum_{i} \sigma_i^z \sigma_{i+1}^z - h \sum_{i} \sigma_i^x
$$

where:
- $J$ is the interaction strength,
- $h$ is the transverse magnetic field,
- $\sigma_i^z$ and $\sigma_i^x$ are the Pauli matrices acting on the $i$-th site.

The TEBD algorithm allows us to simulate the time evolution of the some initial state $|\psi\rangle$ under this Hamiltonian.

## 2. TEBD Algorithm Outline

The TEBD algorithm involves:
1. **Splitting the Hamiltonian** into local terms.
2. **Applying a Trotter decomposition** to approximate the time evolution operator.
3. **Updating the MPS** using local unitary operations.
4. **Truncating the MPS** to control the bond dimension.


## 3. Implementing TEBD Step by Step

### a) Local update
First, we will focus on the local update wherein one 2-body gate is applyed to the MPS. 

The local update consists of 4 steps set out in the course slides (1. Contract, 2. SVD + truncate, 3. Insert identity, 4. Contract). After the local update is complete **2 gamma tensors and 1 lambda matrix** will have been updated.

![](../img/local_update.png)

The function below performs the first step of contracting the sites and 2-qubit gate:

In [2]:
import numpy as np

def contract_sites_and_gate(
    lambda_left: np.ndarray, 
    gamma_left: np.ndarray, 
    lambda_centre: np.ndarray, 
    gamma_right: np.ndarray, 
    lambda_right: np.ndarray,
    gate: np.ndarray,
) -> np.ndarray:
    """
    Contract the two-body gate with the mps site tensors.

    Args:
        lambda_left: The lambda matrix to the left of the left gamma tensor
        gamma_left: The left gamma tensor
        lambda_centre: The lambda matrix in between the two gamma tensors
        gamma_right: The right gamma tensor
        lambda_right: The lambda matrix to the right of the right gamma tensor
        gate: The 2-body gate

    Returns:
        Theta, an order-4 tensor resulting from the contraction.
    """
    theta = np.einsum('ab,bgc,cd,dhe,ef,ghij->aijf',
        np.diagflat(lambda_left), 
        gamma_left, 
        np.diagflat(lambda_centre), 
        gamma_right, 
        np.diagflat(lambda_right), 
        gate
    )
    return theta

**Note: Since the lambda matrices are diagonal, they can be stored as a 1-dimension array containing the diagonal elements only.**

Next, we carry out step 2: splitting the resulting tensor with an SVD and truncation. For simplicity we will specify the maximum number of singlar values to retain after the SVD - this will determine the maximum value for the bond dimension of the MPS. We could have instead specified the truncation error.

Any very small singular values (<10^-14) we remove, otherwise they will cause problems when we need to compute $\lambda^{-1}$.

In [3]:
from typing import Tuple

def perform_svd_and_truncate(
    theta: np.ndarray,
    max_bond_dimension: int,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Splits the theta tensor via an SVD and performs a truncation.
    
    Args:
        theta: The order-4 tensor
        max_bond_dimension: The maximum number of singlar values to keep after the truncation, 
            this will determine the maximum value for the bond dimension of the MPS

    Returns:
        The order-3 tensor U, the updated lambda_centre matrix, the order-3 tensor Vdagger
    """
    D1, d1, d2, D2 = theta.shape
    U, s, Vt = np.linalg.svd(np.reshape(theta, [D1*d1, d2*D2]))
    
    # Choose the number of singular values to keep: 
    # We keep at most max_bond_dimension singular values and truncate singular values smaller than 10**-14
    
    # If we don't remove small singular values here we'll have 
    # problems when we come to inverting the lambda matrix
    r = min(max_bond_dimension, np.count_nonzero(s >= 10**-14))
    
    U = np.reshape(U[:,:r], [D1, d1, r])
    Vt = np.reshape(Vt[:r, :], [r, d2, D2])
    return U, s[:r], Vt

Then, we perform steps 3 and 4.

In [4]:
def insert_identity_and_contract(
    lambda_left: np.ndarray,
    U: np.ndarray,
    lambda_centre_updated: np.ndarray,
    Vdagger: np.ndarray,
    lambda_right: np.array,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Contracts the inverted left and right lambda matrices with U and Vdagger to obtain the updated gamma tensors.

    Args:
        lambda_left: The lambda matrix to the left of the left gamma tensor
        U: The U tensor returned by `perform_svd_and_truncate`
        lambda_centre_updated: The updated lambda_centre matrix returned by `perform_svd_and_truncate`
        Vdagger: The Vdagger tensor returned by `perform_svd_and_truncate`
        lambda_right: The lambda matrix to the right of the right gamma tensor

    Returns:
        The updated gamma_left tensor, the updated gamma_right_tensor
    """
    lambda_right_inverted = np.linalg.inv(np.diagflat(lambda_right))
    lambda_left_inverted = np.linalg.inv(np.diagflat(lambda_left))
    gamma_left_updated = np.einsum('ab,bcd->acd', lambda_left_inverted, U)
    gamma_right_updated = np.einsum('abc,cd->abd', Vdagger, lambda_right_inverted)
    return gamma_left_updated, lambda_centre_updated, gamma_right_updated

Finally, let's combine all those steps into a function for the full local update.

In [5]:
def local_update(
    lambda_left: np.ndarray, 
    gamma_left: np.ndarray, 
    lambda_centre: np.ndarray, 
    gamma_right: np.ndarray, 
    lambda_right: np.ndarray,
    gate: np.ndarray,
    max_bond_dimension: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Performs the full local update.

    Args:
        lambda_left: The lambda matrix to the left of the left gamma tensor
        gamma_left: The left gamma tensor
        lambda_centre: The lambda matrix in between the two gamma tensors
        gamma_right: The right gamma tensor
        lambda_right: The lambda matrix to the right of the right gamma tensor
        gate: The 2-body gate
        max_bond_dimension: The maximum bond dimension of the MPS

    Returns:
        The updated gamma_left tensor, the updated lambda_centre matrix, the updated gamma_right tensor
    """
    theta = contract_sites_and_gate(lambda_left, gamma_left, lambda_centre, gamma_right, lambda_right, gate)
    UsVt = perform_svd_and_truncate(theta, max_bond_dimension)
    gamma_left_updated, lambda_centre_updated, gamma_right_updated = insert_identity_and_contract(lambda_left, *UsVt, lambda_right)
    return gamma_left_updated, lambda_centre_updated, gamma_right_updated
    

### b) Global update

![](../img/global_updates.png)

Now that we have a single local update we want to apply this function for every 2-qubit gate. We will write a function that applies the local update for all odd gates (gates whose left leg acts on a site whose index is odd), followed by all even gates (gates whose left leg acts on a site whose index is even). After one call to this function the MPS will have evolved by one time step dt.

The function will apply all odd/even gates in series (one after the other). However note that each odd/even gate can be applied independently of all other odd/even gates, meaning that they could be applied in **parallel**.

In [ ]:
def global_update(
    lambdas: list,
    gammas: list,
    odd_gates: list,
    even_gates: list,
    max_bond_dimension: int,
) -> Tuple[list, list]:
    """
    Perform the global update applying all gates (odd and even) once.

    Args:
        lambdas: A list of length N+1 containing the lambda matrices of the MPS
        gammas: A list of length N containing the gamma tensors of the MPS
        odd_gates: A list containing the odd gates
        even_gates: A list containing the even gates
        max_bond_dimension: The maximum bond dimension of the MPS

    Returns:
        The list of updated lambdas, the list of updated gammas
    """
    nsites = len(gammas)
    for gate, site in zip(odd_gates, range(1, nsites, 2)):
        gammas[site], lambdas[site+1], gammas[site+1] = local_update(
            lambdas[site], 
            gammas[site], 
            lambdas[site+1], 
            gammas[site+1], 
            lambdas[site+2],
            gate,
            max_bond_dimension,
        )
    for gate, site in zip(even_gates, range(0, nsites, 2)):
        gammas[site], lambdas[site+1], gammas[site+1] = local_update(
            lambdas[site], 
            gammas[site], 
            lambdas[site+1], 
            gammas[site+1], 
            lambdas[site+2],
            gate,
            max_bond_dimension,
        )
    return lambdas, gammas

### c) Make Gates

Now that we have the global update function we now need to construct the gates.